# Funções de janela para análise de dados
<p><img src = https://sp-ao.shortpixel.ai/client/q_glossy,ret_img,w_780,h_450/https://blog.academiain1.com.br/wp-content/uploads/2017/08/104629-big-data-voce-conhece-os-4-tipos-de-analise-de-dados-780x450.jpg width = 500></p>

## Introdução
No módulo anterior, discutimos funções agregadas, funções que podem receber um grande grupo de linhas e gerar um único valor para elas. Muitas vezes, ser capaz de resumir um grupo de linhas em um único valor é importante e útil. No entanto, há momentos em que você deseja manter as linhas individuais, além de obter um valor resumido. Para fazer isso, neste módulo, apresentaremos um novo conjunto de funções chamadas **funções de janela**, que podem calcular estatísticas agregadas mantendo linhas individuais. Estas funções são muito úteis para poder calcular novos tipos de estatísticas, como classificações e médias móveis, com relativa facilidade no SQL. Neste módulo, aprenderemos o que são funções de janela e como podemos usá-las para calcular estatísticas.

## Funções da janela
As funções agregadas nos permitem pegar muitas linhas e converter essas linhas em um número. Por exemplo, a função **COUNT** recebe as linhas de uma tabela e retorna o número de linhas que existem. No entanto, às vezes queremos poder calcular várias linhas, mas ainda manter todas as linhas após o cálculo. Por exemplo, digamos que você queira classificar todos os usuários em ordem de acordo com o momento em que se tornaram clientes, com o primeiro cliente sendo classificado como 1, o segundo cliente mais antigo classificado como 2 e assim por diante. Você pode obter todos os clientes usando a seguinte consulta:

In [2]:
import pandas as pd
import psycopg2 as pg

# Realizando a conexão local no Banco de dados (SQLDA)
connection = pg.connect(user="postgres",
                        password="admin",
                        host="127.0.0.1",
                        port="5432",
                        database="sqlda")
# Query a ser realizada

sql = """
      SELECT *
      FROM customers
      ORDER BY date_added;
      """
df = pd.read_sql_query(con=connection, sql=sql)
df.head()

,customer_id,title,first_name,last_name,suffix,email,gender,ip_address,phone,street_address,city,state,postal_code,latitude,longitude,date_added
0,6173,None,Danila,Gristwood,None,dgristwood4rg@furl.net,F,254.239.58.108,832-157-3870,79865 Hagan Terrace,Katy,TX,77493,29.8678,-95.8298,2010-03-15
1,2625,None,Binky,Dawtrey,None,bdawtrey20w@shareasale.com,M,15.75.236.78,804-990-4322,0353 Iowa Road,Richmond,VA,23208,37.5593,-77.4471,2010-03-15
2,7486,None,Ciro,Ferencowicz,None,cferencowicz5rx@ucoz.ru,M,8.151.167.184,786-458-8754,61 Village Crossing,Miami,FL,33111,25.5584,-80.4582,2010-03-15
3,13390,None,Danika,Lough,None,dloughabx@skype.com,F,188.19.7.207,212-769-6845,38463 Forest Dale Way,New York City,NY,10019,40.7651,-73.9858,2010-03-15
4,12484,None,Lillis,Brayley,None,lbrayley9mr@ovh.net,F,253.224.181.145,None,None,None,None,None,NaN,NaN,2010-03-15


Você pode solicitar clientes do mais antigo ao mais recente, mas não pode atribuir um número a eles. Você pode usar uma função agregada para obter as datas e ordená-las dessa maneira:

In [3]:
sql = """
      SELECT date_added, COUNT(*)
      FROM customers
      GROUP BY date_added
      ORDER BY date_added;
      """
df = pd.read_sql_query(con=connection, sql=sql)
df.head()

,date_added,count
0,2010-03-15,11
1,2010-03-16,13
2,2010-03-17,12
3,2010-03-18,19
4,2010-03-19,23


Embora isso forneça as datas, ele elimina o restante das colunas e ainda não fornece informações de classificação. O que podemos fazer? É aqui que as funções da janela entram em jogo. As funções de janela podem pegar várias linhas de dados e processá-las, mas ainda retêm todas as informações nas linhas. Para coisas como classificações, isso é exatamente o que você precisa. Para melhor compreensão, vamos ver como é uma consulta de função do Windows na próxima seção.

### Noções básicas de funções de janela
A seguir está a sintaxe básica de uma função de janela:

`SELECT {columns},`

`{window_func} OVER (PARTITION BY {partition_key} ORDER BY {order_key})`

`FROM table1;`

Onde **{columns}** são as colunas a serem recuperadas das tabelas para a consulta, **{window_func}** é a função de janela que você deseja usar, **{partition_key}** é a coluna ou colunas nas quais você deseja particionar (mais sobre isso posteriormente), **{order_key}** é a coluna ou colunas pelas quais você deseja ordenar e **table1** é a tabela ou tabelas unidas das quais você deseja extrair dados. A palavra-chave **OVER** indica onde a definição da janela começa.